# Hubert Violet Sedaro Demo Phase 1
## START RECORDING BEFORE DEMO STARTS  
## Big Idea
This script aims to demo Violet platform's ability to auto trigger simulations based on certain parameter changes, and whether the changes result in a successful simulation or not, and relevant validation metrics.
### Use case, I/O, and cutomer's expectation
A Violet customer who makes commercial satellites have some batteries with different capacities and prices. The customer wants to go with the cheapest (and lowest capacity in this case) first. Given the orbit, timeframe, and conditions that this satellite is working in, the customer wants to know if the battery of choice can last through this period. If it does, what is the mimnimal and average state of charge?
### Nomenclature
Agent: Satellites, ground stations, celesial bodies, or any active actor that influences others. \
Templated Agent: It has high Fidelity, and we can study it extensively, such as power and thermal. The main character of the show. \
Scenario: The theatre that all the agents act in. \
Branch: Sedaro simulation is git controlled, so you can create multiple version of a scenario or agent, which are called branches. \



In [105]:
# We will start with defining all the IDs and import all modules.
from sedaro import SedaroApiClient
from pprint import pprint
API_KEY = "PKDqMrtcTK4plJgL7qVlQD.0xQph1lyKzd-pV0-ZL7bRIH7BX54Yjqbz6tluwut3Hvp8XE-RVbfHSz2o5vC77scUhg2xBFuBybplxY6FyXXMQ"
TEMP_AGENT_REPO_BRANCH_ID = "PRkHJlBrpwLDCqPqfbkdrb" # Specific Repo of the Templated agent 
SCENARIO_BRANCH_VERSION_ID = "PRx5rSwrGfkK4n9vFXmVbt" # Version 2

In [106]:
# Define modules
sedaro = SedaroApiClient(api_key = API_KEY)
# agent template is only for satellite model (agent), and don't copy the model repo ID, only the branch ID (version)
agent_template_branch = sedaro.agent_template(TEMP_AGENT_REPO_BRANCH_ID) # we are going to change this templated agent


In [97]:

# NO NEED TO RUN THIS SINCE IT CHANGES REACTION WHEELS' PARAMETERS. IRRELAVENT TO THIS DEMO BUT COULD TO SHOW CAPABILITY

# # # to successfully update a model mass, use this method below. only get the branch ID (version x)
# RCS_Z_id = "PRx7YwymYJgtlTDBjPKfJG"
# RCS_Y_id = "PRybmF9qkFSZFVf2gxYSk5"
# RCS_X_id = "PRybm3zT77x3kCSSYHCKgG"
# RCS_ids = [RCS_X_id, RCS_Y_id, RCS_Z_id]

# for i in range(len(RCS_ids)):
#     RCS = agent_template_branch.block(RCS_ids[i]) # directly use "id" in Blueprint
#     RCS.update(ratedMomentum = 0.8, ratedTorque = 0.2)
#     print("Current RCS rated momentum: ",RCS.ratedMomentum, " , rated torque: ",RCS.ratedTorque)

# # pprint(components.data)

# # Note for Lael: where do i see all the variable names? For example, wheel inertia is inertia, how am I supposed to know that. do i
# # always need to print it out using .data command

# # how does the sun tracking surface work?


In [ ]:
# Version 1 (success)
battery_capacity_Ah = 3.35
num_bat = 3

# Version 2 (Failed)
battery_capacity_Ah = 0.335
num_bat = 3

battery_cell_id = 'PK3PCpCJRn6LpwvhWzNtsb' 
battery_pack_id = 'PRx5qGqQD59tCW4V9tBGQb'
battery_cell_block = agent_template_branch.block(battery_cell_id) # directly use "id" in Blueprint
battery_pack_block = agent_template_branch.block(battery_pack_id) 
battery_cell_block.update(capacity = battery_capacity_Ah)
print('Current battery capacity: ', battery_cell_block.capacity)
battery_pack_block.update(numParallel = num_bat)
print('Current battery in parallel: ', battery_pack_block.numParallel)


## grab all data of spacecraft and repopulate another model

Current battery capacity:  0.335
Current battery in parallel:  3


In [108]:
# Get a scenario branch
scenario_branch = sedaro.scenario(SCENARIO_BRANCH_VERSION_ID)
sim = scenario_branch.simulation

Pull up Sedaro window before starting the simulation

In [112]:
simulation_handle = sim.start(wait=False)
print(simulation_handle['status'])

simulation_handle = sim.status() # must first grab the status to then call the status

print(simulation_handle.poll()) # this synchronously update with the progress bar
# print(simulation_handle.results())


### or we can have a line-based output to check progress
# # 1) Start (but don’t wait for it to finish)
# handle = sim.start(wait=False)

# # 2) At any point, call status()
# status = handle.status()   # same as sim.status(job_id=handle["id"])
# print(f"Status:   {status['status']}")
# print(f"Progress: {status.get('percentComplete', status.get('progress', 'N/A'))}%")

# # 3) Loop until it’s done
# import time
# while status["status"] not in ("SUCCEEDED"): #, "FAILED", "TERMINATED"):
#     time.sleep(1)
#     status = handle.status()
#     print(f"→ {status['status']} ({status.get('percentComplete', '0')}% complete)") # need to change 0 to an output


PENDING


Show in violet for scripts how we can connect the platform with the script + workflow
Discuss Sedaro Blueprint and how that helps with changing variables. 


In [ ]:
stat = sim.stats()
sim_results = simulation_handle.results()
# print(stat['PRkHzzmM9MMY9JPfBDd7LC/0'].keys())
# print(stat['PRkHzzmM9MMY9JPfBDd7LC/1'].keys())
# print(stat['PRknkFRHNYJhkCV8b53CLG/0'].keys())
# print(stat['PRkHzzmM9MMY9JPfBDd7LC/0']['orbitNumber'])
pprint(stat)

{'PRkHzzmM9MMY9JPfBDd7LC/0': {'orbitNumber': {'absAvg': 0.3949342886735274,
                                              'average': 0.3949342886735274,
                                              'integral': 0.020464081251702737,
                                              'max': 1,
                                              'min': 0,
                                              'negativeMax': 0,
                                              'positiveMax': 1},
                              'orbitalElements.a': {'absAvg': 6678.13233691405,
                                                    'average': 6678.13233691405,
                                                    'integral': 346.03691467570735,
                                                    'max': 6678.26719681355,
                                                    'min': 6677.93515147931,
                                                    'negativeMax': None,
                                                    'p

In [114]:
end_time = sim_results.end_time
print("end frame: ",end_time)
sim_status = simulation_handle.status()['status']
print("Sim status: ",sim_status)
average_SOC = stat['PRkHzzmM9MMY9JPfBDd7LC/2']['NSghWfrUj9OyAK8OBAXa-.soc']['absAvg']
min_SOC = stat['PRkHzzmM9MMY9JPfBDd7LC/2']['NSghWfrUj9OyAK8OBAXa-.soc']['min']
print('Average SOC: ', average_SOC)
print('Min SOC: ',min_SOC)

end frame:  2025-06-30 21:21:03.310000
Sim status:  FAILED
Average SOC:  0.5098346603837014
Min SOC:  0.0006828450646829915


At this point, if the simulation failed the workflow will be set to send an email with the summary statistics to remind the customer that this configuration might not owrk.

In [102]:
# investigated_agent_id = 'PRkHzzmM9MMY9JPfBDd7LC' # main agent template
# investigated_agent = sim_results.agent(investigated_agent_id)
# energy_storage_block = scenario_branch.block('NZCSzqYT8iR1rfstDw3AF') # directly use "id" in Blueprint
# pprint(energy_storage_block.data) 
# this block useless so far as it only probes into the block structure


## Challenges and Future Directions
Challenges: to find the specific thing to change seems to be a custom id, will look into more generalized solution; customers still need to be very fluent in using sedaro python client to make this work; we can assist by adding parameters where they can put in ids so no need to insert in the code. Violet scripts and parameter functions need to be improved to ensure reliability; another note that I cannot change the parameter set since i have no access.\
\
Future plans: short term to add in timeframe that user can change simulation timeframe and the power of each reaction wheel; when violet platform supports, will implement email action. mid term goal is to add in thermal analysis. 


ask for a list of things you can access in UI 